In [1]:
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.datasets import load_diabetes
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# TrailMake A

In [6]:
%cd Z:\PPMI_Data\Excels\NonMotors\The251\GeneCpG\TrailMakeA
data = pd.read_csv('TrailMakeA_with_MethylomeDetails.csv')
data.head(1)

Z:\PPMI_Data\Excels\NonMotors\The251\GeneCpG\TrailMakeA


,Sentrix,PATNO,Gender,Age,AgeCate,HYS,APPRDX,Trail_Making_A,cg08012045,cg08300930,...,cg00956423,cg14462343,cg07481734,cg22394535,cg13044444,cg23223756,cg16870358,cg04754776,cg00154746,cg06155771
0,200973410159_R03C01,3000,2,69.1,4,0,2,0.0,0.964598,0.960974,...,0.148515,0.036822,0.053052,0.044051,0.040994,0.047501,0.04635,0.904242,0.021033,0.813738


In [7]:
X = data.drop(['Sentrix','PATNO','HYS','Gender','Age','AgeCate','APPRDX','Trail_Making_A'], axis = 1)
y = data['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 
print(X.head(2))

   cg08012045  cg08300930  cg00134295  cg10537824  cg04990650  cg03379001  \
0    0.964598    0.960974    0.241886    0.852113    0.765526    0.856057   
1    0.962437    0.964442    0.243530    0.826172    0.845330    0.852316   

   cg02215627  cg06699201  cg18100935  cg19220149  ...  cg00956423  \
0    0.874673    0.798285     0.89382    0.879652  ...    0.148515   
1    0.871498    0.653140     0.89167    0.874343  ...    0.109870   

   cg14462343  cg07481734  cg22394535  cg13044444  cg23223756  cg16870358  \
0    0.036822    0.053052    0.044051    0.040994    0.047501    0.046350   
1    0.056582    0.064137    0.053802    0.083556    0.042638    0.041733   

   cg04754776  cg00154746  cg06155771  
0    0.904242    0.021033    0.813738  
1    0.896521    0.033285    0.753096  

[2 rows x 87718 columns]


In [8]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [9]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.6382978723404256

In [10]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100
Iteration: 10 / 100
Iteration: 11 / 100
Iteration: 12 / 100
Iteration: 13 / 100
Iteration: 14 / 100
Iteration: 15 / 100
Iteration: 16 / 100
Iteration: 17 / 100
Iteration: 18 / 100
Iteration: 19 / 100
Iteration: 20 / 100
Iteration: 21 / 100
Iteration: 22 / 100
Iteration: 23 / 100
Iteration: 24 / 100
Iteration: 25 / 100
Iteration: 26 / 100
Iteration: 27 / 100
Iteration: 28 / 100
Iteration: 29 / 100
Iteration: 30 / 100
Iteration: 31 / 100
Iteration: 32 / 100
Iteration: 33 / 100
Iteration: 34 / 100


BorutaPy finished running.

Iteration: 	35 / 100
Confirmed: 	4
Tentative: 	0
Rejected: 	87714


BorutaPy(estimator=RandomForestClassifier(n_estimators=28,
                                          random_state=RandomState(MT19937) at 0x23284997340),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x23284997340, verbose=1)

In [11]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  4


In [12]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [13]:
Top50 = selected_rf_features.iloc[:51, :]
Top50.head(3)

,CpG,Ranking
79610,cg12683454,1
57541,cg03972596,1
16055,cg09252204,1


In [14]:
Top50.to_csv ('TrailMakeA_Boruta_Top50.csv', index = False)
selected_rf_features.to_csv ('TrailMakeA_Boruta_All.csv', index = False)

In [15]:
# create a list of column names to subset from data
col_names = Top50.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['APPRDX'] = data['APPRDX']
df_subset.to_csv ('TrailMakeA_Boruta_Top50_Data.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_972\1836242874.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['APPRDX'] = data['APPRDX']


,cg17427302,cg15313666,cg20995061,cg24465448,cg12696505,cg01083159,cg15629915,cg09252204,cg11492856,cg02479278,...,cg07667731,cg20567867,cg12683454,cg04888085,cg09304381,cg25911279,cg23811160,cg08810486,cg17464366,APPRDX
0,0.933776,0.909546,0.041305,0.023223,0.031196,0.285636,0.054835,0.363327,0.921944,0.677445,...,0.016602,0.923635,0.020790,0.016866,0.981877,0.035425,0.185028,0.446588,0.360898,2
1,0.897221,0.907550,0.022731,0.016847,0.090758,0.323545,0.067318,0.026972,0.911869,0.641669,...,0.013778,0.920654,0.023826,0.014670,0.960567,0.031463,0.171493,0.028135,0.040671,1


# Dream

In [16]:
%cd Z:\PPMI_Data\Excels\NonMotors\The251\GeneCpG\Dream
data = pd.read_csv('Dream_with_MethylomeDetails.csv')
data.head(1)

Z:\PPMI_Data\Excels\NonMotors\The251\GeneCpG\Dream


,Sentrix,PATNO,Gender,Age,AgeCate,HYS,APPRDX,REM_Dream,cg07920064,cg27486695,...,cg12444710,cg01442833,cg17429457,cg26703873,cg21642591,cg06818627,cg20495247,cg12696399,cg22978941,cg12009719
0,200991620021_R03C01,3002,2,67.6,4,2,1,2,0.023631,0.010635,...,0.865247,0.026656,0.916954,0.913505,0.03158,0.041244,0.052317,0.022932,0.045476,0.586523


In [17]:
X = data.drop(['Sentrix','PATNO','HYS','Gender','REM_Dream','Age','AgeCate','APPRDX'], axis = 1)
y = data['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 
print(X.head(2))

   cg07920064  cg27486695  cg08290072  cg19964303  cg24775454  cg26544277  \
0    0.023631    0.010635    0.022312    0.979811    0.023961    0.020120   
1    0.011070    0.008845    0.012105    0.980551    0.015579    0.014584   

   cg23153680  cg06716437  cg18014247  cg19412109  ...  cg12444710  \
0    0.115778    0.190787    0.038345    0.173499  ...    0.865247   
1    0.087679    0.145042    0.035543    0.223806  ...    0.775811   

   cg01442833  cg17429457  cg26703873  cg21642591  cg06818627  cg20495247  \
0    0.026656    0.916954    0.913505    0.031580    0.041244    0.052317   
1    0.021784    0.776223    0.926893    0.019075    0.018032    0.027572   

   cg12696399  cg22978941  cg12009719  
0    0.022932    0.045476    0.586523  
1    0.026038    0.022241    0.526541  

[2 rows x 639 columns]


In [18]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [19]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.6875

In [20]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100
Iteration: 10 / 100
Iteration: 11 / 100
Iteration: 12 / 100
Iteration: 13 / 100
Iteration: 14 / 100
Iteration: 15 / 100
Iteration: 16 / 100
Iteration: 17 / 100
Iteration: 18 / 100
Iteration: 19 / 100
Iteration: 20 / 100
Iteration: 21 / 100
Iteration: 22 / 100
Iteration: 23 / 100
Iteration: 24 / 100
Iteration: 25 / 100
Iteration: 26 / 100
Iteration: 27 / 100
Iteration: 28 / 100
Iteration: 29 / 100
Iteration: 30 / 100
Iteration: 31 / 100
Iteration: 32 / 100
Iteration: 33 / 100
Iteration: 34 / 100
Iteration: 35 / 100
Iteration: 36 / 100
Iteration: 37 / 100
Iteration: 38 / 100
Iteration: 39 / 100
Iteration: 40 / 100
Iteration: 41 / 100
Iteration: 42 / 100
Iteration: 43 / 100
Iteration: 44 / 100
Iteration: 45 / 100
Iteration: 46 / 100
Iteration: 47 / 100
Iteration: 48 / 100
Iteration: 49 / 100
Iteration: 50 / 100
Iteration

BorutaPy(estimator=RandomForestClassifier(n_estimators=24,
                                          random_state=RandomState(MT19937) at 0x23284997D40),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x23284997D40, verbose=1)

In [21]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  3


In [22]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [23]:
Top50 = selected_rf_features.iloc[:51, :]
Top50.head(3)

,CpG,Ranking
563,cg04201744,1
466,cg27068143,1
117,cg02328545,1


In [24]:
Top50.to_csv ('Dream_Boruta_Top50.csv', index = False)
selected_rf_features.to_csv ('Dream_Boruta_All.csv', index = False)

In [26]:
# create a list of column names to subset from data
col_names = Top50.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['PATNO'] = data['PATNO']
df_subset['APPRDX'] = data['APPRDX']
df_subset.to_csv ('Dream_Boruta_Top50_Data.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_972\3187581813.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['PATNO'] = data['PATNO']
C:\Users\system 4\AppData\Local\Temp\ipykernel_972\3187581813.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['APPRDX'] = data['APPRDX']


,cg07916323,cg25836061,cg12175949,cg11361387,cg27521571,cg24164157,cg04856117,cg12728623,cg02328545,cg17403609,...,cg04201744,cg07308881,cg11780096,cg14276366,cg25745663,cg19258682,cg02803589,cg21642591,PATNO,APPRDX
0,0.083608,0.880723,0.019869,0.826294,0.554459,0.104010,0.585446,0.893565,0.975067,0.143041,...,0.890941,0.792484,0.750843,0.878788,0.011783,0.025435,0.924742,0.031580,3002,1
1,0.052975,0.928010,0.016260,0.781571,0.658729,0.078016,0.524912,0.907389,0.970260,0.114913,...,0.802491,0.894306,0.593850,0.918059,0.014489,0.027770,0.944254,0.019075,3008,2
